In [ ]:
# 本程式碼已進行模擬處理，所有敏感資訊（如帳號、密碼、使用者姓名、email、Google Sheet ID）已移除。
# 請在實際部署時使用自己的資料，並妥善保護個人隱私與 API 金鑰。

In [32]:
import datetime
import gspread
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import openai
from oauth2client.service_account import ServiceAccountCredentials

### 設定並連接 Google Sheets API

In [ ]:

SCOPES = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]


SERVICE_ACCOUNT_FILE = "C:\\Users\\vanes\\Downloads\\institutional-care-ab4817c0a31d.json" 

creds = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_FILE, SCOPES)
client = gspread.authorize(creds)


sheet =  # 用你自己的 Sheet ID 替換
sheet = client.open_by_key("# YOUR_SHEET_ID>")

response=   sheet.worksheet('表單回應 1')

# 讀取數據
data= pd.DataFrame(response.get_all_records())


### 發送郵件

In [ ]:
# 📌 Google Form 連結（請填入你的表單網址）

GOOGLE_FORM_URL = #表單連結

# 📧 Gmail 設定
SENDER_EMAIL = #信箱
SENDER_PASSWORD = #密碼


# 📅 取得當月份名稱（e.g., "March"）
month_name = datetime.datetime.now().strftime("%B")
from email.header import Header
# ✉️ 發送郵件
def send_email(to_email, name):
    msg = MIMEMultipart()
    msg["From"] = SENDER_EMAIL
    msg["To"] = to_email
    msg["Subject"] = Header(f"【{month_name}】心理健康調查表填寫通知", 'utf-8')

    # 📝 郵件內容
    body = f"""
親愛的 {name}，

這是本月的機構養育和心智發展關聯性的問卷，請點擊下方連結填寫：
🔗 {GOOGLE_FORM_URL}

請依照自己想法或感覺符合句子描述的程度來填答。
您的回饋對我們非常重要，謝謝您的參與！
    
祝好，
國立政治大學輔導與諮商碩士學位學程 
    """
    msg.attach(MIMEText(body, "plain", _charset="utf-8"))

    
    try:
        # 📨 連接 Gmail 並發送
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.ehlo('localhost')
            server.login(SENDER_EMAIL, SENDER_PASSWORD)
            server.sendmail(SENDER_EMAIL, to_email, msg.as_string())
        print(f"郵件已成功發送給 {name}")
    except Exception as e:
        print(f"發送郵件時出現錯誤: {e}")

# 🔄 發送給所有受試者
for person in info_data:
    email = person['Email Address'] 
    name = person['Name']
    send_email(email , name)

print("📧 本月心理健康表單已成功發送給所有受試者！")



'\nGOOGLE_FORM_URL = "https://forms.gle/Jp4HShDxykiEAp4s9"\n\n# 📧 Gmail 設定\nSENDER_EMAIL = "christine921117@gmail.com"\nSENDER_PASSWORD = "fxzmflqpblyiyeeg"\n\n\n# 📅 取得當月份名稱（e.g., "March"）\nmonth_name = datetime.datetime.now().strftime("%B")\nfrom email.header import Header\n# ✉️ 發送郵件\ndef send_email(to_email, name):\n    msg = MIMEMultipart()\n    msg["From"] = SENDER_EMAIL\n    msg["To"] = to_email\n    msg["Subject"] = Header(f"【{month_name}】心理健康調查表填寫通知", \'utf-8\')\n\n    # 📝 郵件內容\n    body = f"""\n親愛的 {name}，\n\n這是本月的機構養育和心智發展關聯性的問卷，請點擊下方連結填寫：\n🔗 {GOOGLE_FORM_URL}\n\n請依照自己想法或感覺符合句子描述的程度來填答。\n您的回饋對我們非常重要，謝謝您的參與！\n\n祝好，\n國立政治大學輔導與諮商碩士學位學程 \n    """\n    msg.attach(MIMEText(body, "plain", _charset="utf-8"))\n\n\n    try:\n        # 📨 連接 Gmail 並發送\n        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:\n            server.ehlo(\'localhost\')\n            server.login(SENDER_EMAIL, SENDER_PASSWORD)\n            server.sendmail(SENDER_EMAIL, to_email, msg.as_string())\n        

### 計算心理健康指數 

In [35]:
def calculate_BSRS(row):
    health_index = sum(row[['1. 最近一週會不會「覺得緊張或不安」？', 
                            '2. 最近一週會不會「覺得容易苦惱或動怒」？', 
                            '3. 最近一週會不會「感覺憂鬱、心情低落」？', 
                            '4. 最近一週會不會「覺得比不上別人」？', 
                            '5. 最近一週會不會「覺得睡眠困難，譬如難以入睡、易醒或早醒」？']]) 
    
    
    
    if pd.to_numeric(row['6. 這一個月來，有自殺的想法？'], errors='coerce') >= 2:
       referral_advice = (
        "愛惜生命、經營屬於自己的美好生活\n\n"
        "❣ 重要提醒：你的回應顯示你最近有較強烈的自我傷害意念，這是一個特別需要關注的訊號，建議你務必尋求以下諮商資源的協助，若有需要，歡迎你與我們聯絡（pywu@nccu.edu.tw)。\n\n"
        "❣ 請不要獨自承擔這些情緒，你並不孤單！找一位值得信任的親友談談，或考慮尋求心理專業人士的協助，請參考以下諮商資源。\n\n"
        "❣ 如果你的情緒非常低落，甚至有強烈的自我傷害或自殺衝動，請立即尋求幫助。\n\n"
        "❣ 請撥打 1925📞（台灣安心專線），或尋找最近的心理健康資源，專業人士可以幫助你度過這段困難時期。\n\n"
        "❣ 這些情緒並不代表你的全部，請給自己一個機會，你值得被幫助，也值得擁有更好的生活！💛\n"
)

    
    else:
        referral_advice = ""
        
    if health_index >=15:
        BSRS_feedback =  "你的情緒狀況目前非常低落，可能對日常生活造成了嚴重的影響。這樣的情緒狀態不容忽視，建議立刻尋求心理治療或精神科專業協助。\n" \
                         "你值得被幫助，並且值得過更好的生活。別猶豫，採取行動來改善情況，讓自己重拾希望💔"

    elif 10 <= health_index < 15:
        BSRS_feedback = "情緒狀況偏低，壓力可能已經對你的生活造成一定影響。這時候，你可能會感到焦慮、疲憊或低落，建議尋求心理諮詢來處理這些情緒。\n" \
                    "試著與信任的朋友或家人溝通，或進行運動來舒解壓力。別忘了，專業的幫助能讓你走出困境💛"
    elif 5 <= health_index < 10:
        BSRS_feedback = "情緒狀態有些波動，可能會開始影響到日常生活，建議你尋求心理諮詢或專業人士的幫助。\n" \
                    "一步一步來，你可以調整自己，讓專業人士協助你找到合適的調適方式💜"
    else: 
        BSRS_feedback = "你的情緒狀態非常穩定，情緒適應良好，請繼續保持目前的生活步調 \n" \
                    "如果你感到稍微疲憊或低落，也不必過度擔心，偶爾的波動是正常的。保持積極的心態，繼續照顧自己！💖"
        
    return health_index, BSRS_feedback, referral_advice
    

data[['HealthIndex', 'BSRS_Feedback', 'ReferralAdvice']] = data.apply(calculate_BSRS, axis=1, result_type='expand')

### 計算 RFQ_U / RFQ_C

In [36]:
import pandas as pd

def calculate_rfq(row):
    rfq_c_weights = [3, 2, 1, 0, 0, 0, 0]  # RFQ-C 評分標準
    rfq_u_weights = [0, 0, 0, 0, 1, 2, 3]  # RFQ-U 評分標準

    # RFQ 題目
    rfq_c_questions = [
        '1. 我猜不出他人的想法。',
        '2. 對於我所做的事情，我並不總是知道為什麼要這樣做。',
        '3. 當我生氣時會說出一些氣話，而我卻不知道為什麼會這麼說。',
        '4. 當我生氣時，我會說一些事後會後悔的話。',
        '5. 如果我對自己沒有把握，我可能會做出冒犯他人的行為。',
        '6. 有時我會做一些不確切知道為什麼要去做的事。'
    ]

    rfq_u_questions = [
        '2. 對於我所做的事情，我並不總是知道為什麼要這樣做。',
        '4. 當我生氣時，我會說一些事後會後悔的話。',
        '5. 如果我對自己沒有把握，我可能會做出冒犯他人的行為。',
        '6. 有時我會做一些不確切知道為什麼要去做的事。',
        '7. 我總是知道我的感受。',
        '8. 強烈的感覺總能擾亂我的情緒。'
    ]

    # 計算 RFQ_C 和 RFQ_U
    rfq_c = sum(rfq_c_weights[v - 1] for v in row.loc[rfq_c_questions]) / len(rfq_c_questions)
    rfq_u = sum(rfq_u_weights[v - 1] for v in row.loc[rfq_u_questions]) / len(rfq_u_questions)

    # 給出個人化回饋
    if rfq_c > 1 and rfq_u >= 2:
        rfq_feedback = "你可能同時容易過度解讀他人的行為，但又對自己的內在狀態不確定。這可能讓你在社交互動時感到焦慮或困惑，建議透過書寫、對話來整理思緒，幫助自己建立更穩定的內在覺察。"
    elif rfq_c > 1 and rfq_u < 2:
        rfq_feedback = "你能夠理解自己的情緒，但可能會過度解讀他人的行為，容易想太多。試著練習放慢思考，不要急著解釋或揣測對方的意圖，給自己一些喘息的空間。"
    elif rfq_c <= 1 and rfq_u >= 2:
        rfq_feedback = "你不會過度解讀他人，但可能較難理解自己的情緒和行為模式。這可能讓你在面對壓力時較難做出適當調適，建議透過日記或與信任的人討論來提升自我覺察能力。"
    else:
        rfq_feedback = "你的心智化能力較為平衡，能夠適當理解自己的情緒，且不會過度解讀他人的行為。這是一個穩定的狀態，持續關注自己的心理健康，適時調整即可！"

    return pd.Series([rfq_c, rfq_u, rfq_feedback])

# **步驟 1：計算 RFQ_C、RFQ_U、個人回饋**
data[['RFQ_C', 'RFQ_U', 'RFQ_Feedback']] = data.apply(calculate_rfq, axis=1)

# **步驟 2：計算當月份的平均值**
rfq_monthly_avg = data.groupby("這次填寫的是幾月份的記錄")[['RFQ_C', 'RFQ_U']].transform("mean")

# **步驟 3：合併回原 DataFrame**
data[['RFQ_C_Mean', 'RFQ_U_Mean']] = rfq_monthly_avg


### 計算個人困擾

In [37]:
dimensions = {
    "生涯相關困擾": ["請依個人感受，點選符合您情形的數字。 [生涯困擾]", "請依個人感受，點選符合您情形的數字。 [學習困擾]"],
    '情緒與心理健康困擾': ["請依個人感受，點選符合您情形的數字。 [情緒困擾]", "請依個人感受，點選符合您情形的數字。 [精神困擾]"],
    "人際關係困擾": ["請依個人感受，點選符合您情形的數字。 [人際困擾]", "請依個人感受，點選符合您情形的數字。 [感情困擾]", "請依個人感受，點選符合您情形的數字。 [家庭困擾]"],
    "生活與時間管理困擾": ["請依個人感受，點選符合您情形的數字。 [時間管理]", "請依個人感受，點選符合您情形的數字。 [生活困擾]"],
    "生理健康困擾": ["請依個人感受，點選符合您情形的數字。 [生理困擾]"]
}
# 計算個人平均分數
def calculate_personal_scores(row):
    scores = {}
    for dimension, categories in dimensions.items():
        scores[dimension] = row[categories].mean()  # 計算該指標的平均值
    return pd.Series(scores)

data[['生涯相關困擾','情緒與心理健康困擾', "人際關係困擾","生活與時間管理困擾","生理健康困擾"]]= data.apply(calculate_personal_scores, axis=1)

In [ ]:
# **函數：清理文件或資料夾名稱中的特殊字符，並替換斜線**
import os
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from datetime import datetime

matplotlib.rcParams['font.family'] = 'Microsoft YaHei'

# 目前的年份 & 當前月份
current_year = datetime.now().year
current_month_name = "三月份"  # 修改這個變數來調整目前月份

# 定義月份字典與順序
months_dict = {
    "一月份": "01", "二月份": "02", "三月份": "03", "四月份": "04", "五月份": "05", "六月份": "06",
    "七月份": "07", "八月份": "08", "九月份": "09", "十月份": "10", "十一月份": "11", "十二月份": "12"
}
months = list(months_dict.keys())
current_month_index = months.index(current_month_name)

# **函數：將「這次填寫的是幾月份的記錄」轉為 YYYY-MM**
def convert_to_datetime(month_text):
    month_number = months_dict[month_text]  # 取得月份的數字
    month_index = months.index(month_text)  # 找到月份的索引

    # 決定年份標記
    if month_index >= months.index("十一月份"):  # 11月或12月標記 2024
        year = 2024
    else:  # 1-10月標記 2025
        year = 2025

    return pd.to_datetime(f"{year}-{month_number}", format="%Y-%m")

# 假設你有原始的 data DataFrame，這裡是轉換「記錄時間」
# 假設 data 已經存在，例如：
# data = pd.DataFrame(...)

# **轉換 DataFrame 的「記錄時間」**
data["記錄時間"] = data["這次填寫的是幾月份的記錄"].apply(convert_to_datetime)

# **過濾只包含當前月份及之前的數據**
current_month = pd.to_datetime(f"{current_year}-{months_dict[current_month_name]}", format="%Y-%m")
filtered_data = data[data["記錄時間"] <= current_month]

# **計算全體平均 HealthIndex**
average_trend = filtered_data.groupby("記錄時間")["HealthIndex"].mean().reset_index()

# **產生資料夾來存圖片**
output_folder = "心理困擾傾向趨勢圖"
os.makedirs(output_folder, exist_ok=True)

def clean_name(name):
    # 替換斜線為 "_or_"
    name = name.replace("/", "_or_")
    # 可以根據需要添加更多的字符清理邏輯
    return name

# **函數：繪製個人健康趨勢圖**
def plot_health_trend(person_data, name):
    selected_dates = person_data["記錄時間"]
    selected_health_indexes = person_data["HealthIndex"]

    # **確保數據排序正確**
    sorted_indices = selected_dates.argsort()
    sorted_dates = selected_dates.iloc[sorted_indices]  # 保持 datetime 格式
    sorted_health_indexes = selected_health_indexes.iloc[sorted_indices]

    # **繪製個人趨勢**
    plt.figure(figsize=(10, 6))
    plt.plot(sorted_dates, sorted_health_indexes, marker='o', linestyle='-', 
             color="dodgerblue", label='個人心理困擾傾向指數')

    # **繪製全體平均趨勢**
    plt.plot(average_trend["記錄時間"], average_trend["HealthIndex"], marker='s', linestyle='--', 
             color="orange", label='群體平均心理困擾傾向指數')

    # **圖表設定**
    plt.xticks(sorted_dates, sorted_dates.dt.strftime("%Y-%m"), rotation=45)  # 確保 X 軸是正確的時間順序
    plt.xlabel('月份')
    plt.ylabel('心理困擾傾向指數')
    plt.title(f'{name} 的心理困擾傾向趨勢')
    plt.grid(True)
    plt.legend()

    # **儲存圖表**
    cleaned_name = clean_name(name)  # 清理名稱中的特殊字符
    filename = f"{output_folder}/{cleaned_name}_心理困擾傾向趨勢.png"
    plt.savefig(filename, dpi=300, bbox_inches="tight")
    plt.close()

    print(f"✅ 圖表已儲存: {filename}")


    
for name in filtered_data['我是'].unique():
    person_data = filtered_data[filtered_data['我是'] == name]
    plot_health_trend(person_data, name)

In [39]:
output_folder = "五大指標困擾程度圖"
os.makedirs(output_folder, exist_ok=True)

# **函數：清理檔案名稱，去掉特殊字符**
def clean_name(name):
    # 替換斜線為 "_or_"
    name = name.replace("/", "_or_")
    
    # 可以根據需要添加更多的字符清理邏輯
    return name

# 為每個人生成熱力圖
for person in data["我是"].unique():  # 取得所有人名
    person_data = data[data["我是"] == person].set_index("這次填寫的是幾月份的記錄")[
        ["生涯相關困擾", "情緒與心理健康困擾", "人際關係困擾", "生活與時間管理困擾", "生理健康困擾"]
    ]

    # 繪製熱力圖
    plt.figure(figsize=(8, 6))
    sns.heatmap(person_data.T, annot=True, cmap="Reds", center=5, linewidths=0.5, cbar=True)

    # 標題與格式調整
    plt.title(f"{person} 五大指標困擾程度", fontsize=14)
    plt.xlabel("月份", fontsize=12)
    plt.ylabel("困擾指標", fontsize=12)

    # **清理名稱**
    cleaned_person = clean_name(person)

    # **只存一張圖，不分月份**
    file_name = f"{output_folder}/{cleaned_person}_五大指標困擾程度.png"
    plt.savefig(file_name, dpi=300, bbox_inches="tight")
    plt.close()

print("✅ 所有熱力圖已存入資料夾！")

✅ 所有熱力圖已存入資料夾！


In [41]:
filtered_data.to_csv("all_data.csv", index=False)
filtered_data1 = filtered_data[[
    "我是",
    "您的機構是？",
    "這次填寫的是幾月份的記錄",
    "E-mail ：（請固定填同一個）",
    "HealthIndex",
    "BSRS_Feedback",
    "ReferralAdvice",
    "RFQ_C",
    "RFQ_U",
    "RFQ_Feedback",
    "RFQ_C_Mean",
    "RFQ_U_Mean",
    "生涯相關困擾",
    "情緒與心理健康困擾",
    "人際關係困擾",
    "生活與時間管理困擾",
    "生理健康困擾"
]]

filtered_data1.to_csv("processed_data.csv", index=False)

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from pathlib import Path
import re

def send_psychological_report(name, current_month, health_index, health_feedback, health_referral_advice,
                              rfq_feedback, rfq_c, rfq_u, email,rfq_u_mean,rfq_c_mean):
    
    clean_ = re.sub(r'^\d+\.', '', name) 
    # 準備 email 內容
    email_body = f"""
    <html>
    <body>
    <p>親愛的 <b>{clean_name}</b>，</p>

    <p>感謝您參與{current_month}的心理健康測評，以下是您本月測評結果。我們希望這些資訊能幫助您更了解自己的心理狀況，並在日常生活中找到更多提升心理健康的方式。</p>

    <hr style="border: 2px solid #ccc;" /> 

    <p><b>🌱心理困擾傾向分數：</b><br>
    您的心理困擾傾向分數為 <span style="color: #1e90ff; font-weight: bold;">{health_index}</span><br>
    {health_feedback}<br>
    <hr style="border: 1px dashed #ccc;" />


    
    <p><b style="font-weight: bold; color: #d9534f">❤️‍🩹心理健康建議：</b><br>
    {health_referral_advice.replace("\n", "<br>")}</p>
    
    <hr style="border: 1px dashed  #ccc;" />

    <p><b>🧠心智化指數：</b><br>
    您的過度心智化（RFQ-C）指數為 <span style="color: #1e90ff; font-weight: bold;">{rfq_c:.2f}</span> (全體平均:{rfq_c_mean:.2f})<br> 
    心智化不足（RFQ-U）指數為 <span style="color: #1e90ff; font-weight: bold;">{rfq_u:.2f}</span> (全體平均:{rfq_u_mean:.2f})<br>
    {rfq_feedback}</p>
    
    <hr style="border: 1px dashed #ccc;" />
    
    <p><b>📊 心理困擾傾向趨勢與五大指標困擾程度：</b><br>
    為了幫助您更直觀地了解自己的心理健康狀況，我們製作了一些圖表，展示了您的心理困擾傾向趨勢與五大指標困擾程度變化：</p>

    <ul>
        <li><b>心理困擾傾向趨勢圖</b><br>
        附圖一顯示您的心理困擾傾向指數隨時間的變化，您可以觀察到個人每月的變化，以及群體平均的變化，<b style="font-weight: bold; color: #d9534f">指數越高代表這段時間你深受情緒困擾</b>。有助於你識別特定時期的壓力點或需要注意的地方。
        </li><br>


        <li><b>五大指標困擾程度圖</b><br>
        附圖二顯示您在五個指標的困擾程度（生涯相關困擾、情緒語心理健康困擾、人際關係困擾、生活與時間管理困擾、生理健康困擾）。當方塊格子的<b style="font-weight: bold; color: #d9534f">顏色越深(格子內數字>2)</b>，代表這些指標可能對您的生活品質造成<b style="font-weight: bold; color: #d9534f">更大的負面影響</b>，要多加留意。<br>
        </li>
    </ul>
    
    <hr style="border: 2px solid #ccc;" />  
    
    <p>若有需要，請善用以下心理健康資源：</p>

    <ul>
        <li>
            <b>全國心理健康促進與精神醫療服務資源</b><br>
            <a href="https://dep.mohw.gov.tw/domhaoh/cp-328-8720-107.html" target="_blank">
                https://dep.mohw.gov.tw/domhaoh/cp-328-8720-107.html
            </a>
        </li>
        <br>
        <li>
            <b>「心快活」－心理健康學習平台（查詢各縣市心理諮商與精神醫療資源）</b><br>
            <a href="https://wellbeing.mohw.gov.tw/nor/mmap" target="_blank">
            https://wellbeing.mohw.gov.tw/nor/mmap
            </a>
        </li>
        <br>
        <li>
            <b>2025 全台免費心理諮商資源、24 小時免費心理諮詢專線</b><br>
            <a href="https://clinic.farhugs.com/posts/66c84402f757300001de695b" target="_blank">
            https://clinic.farhugs.com/posts/66c84402f757300001de695b
            </a>
        </li>
    </ul>

    <p>如果您有任何疑問，請隨時聯繫我們，讓我們一起關注並提升您的心理健康💖</p>
    
    
    </body>
    </html>
    """
    if not health_referral_advice.strip():  # 去除前後空白字符
        email_body = re.sub(
            r'<p><b style="font-weight: bold; color: #d9534f">❤️‍🩹心理健康建議：</b><br>.*?<hr style="border: 1px dashed  #ccc;" />',
            "",
            email_body,
            flags=re.DOTALL
        )
    
    
    clean_person=clean_name(name)
    
    # 發送郵件
    sender_email = "christine921117@gmail.com"
    sender_password = "fxzmflqpblyiyeeg"

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = email
    msg['Subject'] = f"{clean_}的心理測評結果 ({current_month})"
    msg.attach(MIMEText(email_body, 'html'))  # HTML內容

    file_path = Path(f"五大指標困擾程度圖/{clean_person}_五大指標困擾程度.png")
    file_path1 = Path(f"心理困擾傾向趨勢圖/{clean_person}_心理困擾傾向趨勢.png")
    
    # 附加圖片檔案
    for path, filename in [(file_path, f"{clean_person}_五大指標困擾程度.png"), (file_path1, f"{clean_person}_心理困擾傾向趨勢.png")]:
        if path.exists() and path.is_file():
            with open(path, "rb") as attachment:
                img_part = MIMEImage(attachment.read())
                img_part.add_header("Content-Disposition", "attachment", filename=filename)
                msg.attach(img_part)
        else:
            print(f"⚠️ 檔案 {filename} 不存在或無法讀取")

    # 發送郵件
    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.ehlo('localhost')
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, email, msg.as_string())

    print(f"郵件已發送給 {name} ({email})")




In [ ]:

current_month = "三月份"

# 篩選出當月的資料
current_month_data = data[data["這次填寫的是幾月份的記錄"] == current_month]



# 迴圈處理每一個當月的數據
for index, entry in current_month_data.iterrows():
    # 準備發送的每個變數
    health_index = entry["HealthIndex"]
    health_feedback = entry["BSRS_Feedback"]
    health_referral_advice = entry["ReferralAdvice"]
    rfq_feedback = entry["RFQ_Feedback"]
    rfq_c = entry['RFQ_C']
    rfq_u = entry['RFQ_U']
    name = entry["我是"]  # 確保欄位名稱正確
    email = entry["E-mail ：（請固定填同一個）"]
    rfq_u_mean = entry['RFQ_U_Mean']
    rfq_c_mean = entry['RFQ_C_Mean']

    # 呼叫 send_psychological_report 函數來發送郵件
    send_psychological_report(name=name, 
                              current_month=current_month, 
                              health_index=health_index, 
                              health_feedback=health_feedback, 
                              health_referral_advice=health_referral_advice,
                              rfq_feedback=rfq_feedback, 
                              rfq_c=rfq_c, 
                              rfq_u=rfq_u, 
                              email=email,
                              rfq_u_mean=rfq_u_mean,
                              rfq_c_mean=rfq_c_mean)
